In [1]:
import os 

In [2]:
storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "main"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [3]:
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("process_sales_silver") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \
    

# set minio config
conf.set("spark.sql.catalog.nessie.s3.endpoint", AWS_S3_ENDPOINT)
# conf.set("spark.sql.catalog.nessie.warehouse", FULL_PATH_TO_WAREHOUSE)
# conf.set("spark.sql.catalog.nessie.uri", NESSIE_URI)
conf.set("spark.sql.catalog.nessie.ref", REF)
conf.set("spark.sql.catalog.nessie.authentication.type", "NONE") 
conf.set("spark.sql.catalog.nessie.s3.access-key-id", AWS_ACCESS_KEY)
conf.set("spark.sql.catalog.nessie.s3.secret-access-key", AWS_SECRET_KEY)
conf.set("spark.sql.catalog.nessie.s3.path-style-access", "true")



In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/11 08:48:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Process sales header

In [22]:
spark.sql("TRUNCATE TABLE nessie.silver.sales_header")

DataFrame[]

In [23]:
spark.sql("""
WITH CTE_SALES AS
(
  select 
      SalesOrderNumber
      , CustomerUsername
      , OrderDate
      , DueDate
      , ShipDate
      , SalesTerritoryRegion
      , Currency
      , source_filepath
      , ingestion_datetime
      , row_number() over(partition by SalesOrderNumber, CustomerUsername, OrderDate, DueDate, ShipDate, SalesTerritoryRegion, Currency order by 1) as rownum
    from nessie.adventureWorks.sales
)
MERGE INTO nessie.silver.sales_header as target
USING (
  select *
  from CTE_SALES
  where rownum = 1
) as source
ON target.SalesOrderNumber = source.SalesOrderNumber
WHEN MATCHED THEN 
UPDATE SET 
  target.SalesOrderNumber = source.SalesOrderNumber
  , target.CustomerUsername = source.CustomerUsername
  , target.OrderDate = source.OrderDate
  , target.DueDate = source.DueDate
  , target.ShipDate = source.ShipDate
  , target.SalesTerritoryRegion = source.SalesTerritoryRegion
  , target.Currency = source.Currency
  , target.source_filepath = source.source_filepath
  , target.ingestion_datetime = source.ingestion_datetime
WHEN NOT MATCHED THEN INSERT
(
  SalesOrderNumber
  , CustomerUsername
  , OrderDate
  , DueDate
  , ShipDate
  , SalesTerritoryRegion
  , Currency
  , source_filepath
  , ingestion_datetime
)
VALUES
(
  source.SalesOrderNumber
  , source.CustomerUsername
  , source.OrderDate
  , source.DueDate
  , source.ShipDate
  , source.SalesTerritoryRegion
  , source.Currency
  , source.source_filepath
  , source.ingestion_datetime
)
""").show()

++
||
++
++



In [24]:
%%sql 

select *
from nessie.silver.sales_header

SalesOrderNumber,CustomerUsername,OrderDate,DueDate,ShipDate,SalesTerritoryRegion,Currency,source_filepath,ingestion_datetime,rownum
SO43697,cole1,2010-12-29,2011-01-10,2011-01-05,Canada,CAD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43698,rachael16,2010-12-29,2011-01-10,2011-01-05,France,FRF,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43699,sydney61,2010-12-29,2011-01-10,2011-01-05,Northwest,USD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43700,ruben10,2010-12-29,2011-01-10,2011-01-05,Southwest,USD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43701,christy12,2010-12-29,2011-01-10,2011-01-05,Australia,AUD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43702,colin45,2010-12-30,2011-01-11,2011-01-06,Southwest,USD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43703,albert7,2010-12-30,2011-01-11,2011-01-06,Australia,AUD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43704,julio1,2010-12-30,2011-01-11,2011-01-06,Australia,AUD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43705,curtis9,2010-12-30,2011-01-11,2011-01-06,Australia,AUD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
SO43706,edward26,2010-12-31,2011-01-12,2011-01-07,Southwest,USD,abfss://datalake@storagesii.dfs.core.windows.net/sales/sales_2010.csv,2025-06-11 09:27:05.371552,None
